In [64]:
import pandas as pd
#estructura de como se almacenan los datos
data = {"Nombre":["Carlos Bermudez","El meza se la come","El papu","John Maxwell"],
        "Genero":["M","F","M","M"],
        "Edad": [23,24,None,23],
       "Pago":[19000,1700,15000,19000],
       "Cargo":["Programador","Puta","Frontend","Programador"]}
#estructuracion de datos
df = pd.DataFrame(data)

In [65]:
df

,Nombre,Genero,Edad,Pago,Cargo
0,Carlos Bermudez,M,23.0,19000,Programador
1,El meza se la come,F,24.0,1700,Puta
2,El papu,M,NaN,15000,Frontend
3,John Maxwell,M,23.0,19000,Programador


Proceso de Preprocesamiento de tuberias:

Dejar el alguna de las columnas
Atribuir Edades
Volver Generos en Binario/Numericos


In [66]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#Metodo para ir dejando columnas de nuestra tabla
df = df.drop(["Nombre"],axis=1)

df

,Genero,Edad,Pago,Cargo
0,M,23.0,19000,Programador
1,F,24.0,1700,Puta
2,M,NaN,15000,Frontend
3,M,23.0,19000,Programador


In [67]:
imputer =  SimpleImputer(strategy="mean")
df['Edad'] = imputer.fit_transform(df[['Edad']])

In [68]:
df

,Genero,Edad,Pago,Cargo
0,M,23.000000,19000,Programador
1,F,24.000000,1700,Puta
2,M,23.333333,15000,Frontend
3,M,23.000000,19000,Programador


In [69]:
#cambiar genero a numerico
genero = {"M":0, "F":1}
df["Genero"] = [genero[g] for g in df["Genero"]]

In [70]:
df

,Genero,Edad,Pago,Cargo
0,0,23.000000,19000,Programador
1,1,24.000000,1700,Puta
2,0,23.333333,15000,Frontend
3,0,23.000000,19000,Programador


In [71]:
encoder = OneHotEncoder()
matrix = encoder.fit_transform(df[['Cargo']]).toarray()

columnas = ["Programador","Puta","Frontend"]
for i in range(len(matrix.T)):
    df[columnas[i]] = matrix.T[i]
df = df.drop(["Cargo"],axis=1)    

In [72]:
df

,Genero,Edad,Pago,Programador,Puta,Frontend
0,0,23.000000,19000,0.0,1.0,0.0
1,1,24.000000,1700,0.0,0.0,1.0
2,0,23.333333,15000,1.0,0.0,0.0
3,0,23.000000,19000,0.0,1.0,0.0


In [77]:
from sklearn.base import BaseEstimator,TransformerMixin
#clase para quitar la columna Nombre
class NameDropper(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        return X.drop(['Nombre'],axis=1)
#clase para ajustar edad   
class AjustarEdad(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
          imputer =  SimpleImputer(strategy="mean")
          X['Edad'] = imputer.fit_transform(X[['Edad']])
          return X

class AjustarCaracteristicas(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
      #cambiar genero a numerico
        genero = {"M":0, "F":1}
        X["Genero"] = [genero[g] for g in X["Genero"]]

        encoder = OneHotEncoder()
        matrix = encoder.fit_transform(X[['Cargo']]).toarray()

        columnas = ["Programador","Mercadotecnia","Diseño"]
        for i in range(len(matrix.T)):
            X[columnas[i]] = matrix.T[i]
            
        return  X.drop(["Cargo"],axis=1)  

In [78]:
Datos = {"Nombre":["Charl Vazquez","Maria Luisa","Alex Shinoda","Ana Arredondo"],
        "Genero":["M","F","M","F"],
        "Edad": [23,24,None,23],
       "Pago":[19000,17000,15000,19000],
       "Cargo":["Programador","Programador","Mercadotecnia","Diseño"]}

dataset = pd.DataFrame(Datos)

In [79]:
dataset
dropper = NameDropper()
dropper.fit_transform(dataset)

,Genero,Edad,Pago,Cargo
0,M,23.0,19000,Programador
1,F,24.0,17000,Programador
2,M,NaN,15000,Mercadotecnia
3,F,23.0,19000,Diseño


In [80]:
dataset
imp = AjustarEdad()
caracteristicas = AjustarCaracteristicas()
caracteristicas.fit_transform(imp.fit_transform(dataset))

,Nombre,Genero,Edad,Pago,Programador,Mercadotecnia,Diseño
0,Charl Vazquez,0,23.000000,19000,0.0,0.0,1.0
1,Maria Luisa,1,24.000000,17000,0.0,0.0,1.0
2,Alex Shinoda,0,23.333333,15000,0.0,1.0,0.0
3,Ana Arredondo,1,23.000000,19000,1.0,0.0,0.0


In [81]:
Datos = {"Nombre":["Charl Vazquez","Maria Luisa","Alex Shinoda","Ana Arredondo"],
        "Genero":["M","F","M","F"],
        "Edad": [23,24,None,23],
       "Pago":[19000,17000,15000,19000],
       "Cargo":["Programador","Programador","Mercadotecnia","Diseño"]}

dataset = pd.DataFrame(Datos)

In [82]:
dataset

,Nombre,Genero,Edad,Pago,Cargo
0,Charl Vazquez,M,23.0,19000,Programador
1,Maria Luisa,F,24.0,17000,Programador
2,Alex Shinoda,M,NaN,15000,Mercadotecnia
3,Ana Arredondo,F,23.0,19000,Diseño


In [88]:
#metodo de pipeline por sklearn
from sklearn.pipeline import Pipeline

tuberia = Pipeline([
    ('dejar',NameDropper()),
    ('imponer',AjustarEdad()),
    ('encoder',AjustarCaracteristicas())
])

tuberia.fit_transform(dataset)

,Genero,Edad,Pago,Programador,Mercadotecnia,Diseño
0,0,23.000000,19000,0.0,0.0,1.0
1,1,24.000000,17000,0.0,0.0,1.0
2,0,23.333333,15000,0.0,1.0,0.0
3,1,23.000000,19000,1.0,0.0,0.0
